# March Madness 2024 Code
### Team: Taylor Last, Ted Woodsides, Jake Hopkins

### Notes
- **Please add docstrings or commments to code so we all know what's going on**
- I can't find how to all work on the same notebook so we might have to copy code and all work on separate pieces
- **Check create_submission and simulate functions to see how to get a bracket in the form the competition asks for**
### Things to keep in mind
- **Days are standardized already**: Dayzero tells you the date corresponding to DayNum=0 during that season. All game dates have been aligned upon a common scale so that (each year) the Monday championship game of the men's tournament is on DayNum=154. Working backward, the men's national semifinals are always on DayNum=152, the "play-in" games are on days 134-135, men's Selection Sunday is on day 132, the final day of the regular season is also day 132, and so on. 
- **Special note about "Season" numbers**: the college basketball season lasts from early November until the national championship tournament that starts in the middle of March. For instance, this year the first regular season games were played in November 2023 and the national championship games will be played in April 2024. Because a basketball season spans two calendar years like this, it can be confusing to refer to the year of the season. By convention, when we identify a particular season, we will reference the year that the season ends in, not the year that it starts in. So for instance, the current season will be identified in our data as the 2024 season, not the 2023 season or the 2023-24 season or the 2023-2024 season, though you may see any of these in everyday use outside of our data.

### Questions to consider
- Do we want to train the model by making prediction for all games (regular season and post), then only make predictions for tournament games, or do we want to only train on tournament games?

### Featues I'd like to add
- Quantify guard play
- Free throw percentage
- Pace
- Time of possession

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from typing import List
from tqdm import tqdm
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost

import networkx
from networkx.algorithms.traversal.depth_first_search import dfs_edges

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Refer to this if you need to look up documentation
print(f"Pandas Version: {pd.__version__}")

Pandas Version: 2.1.4


In [3]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
data = dict()
for dirname, _, filenames in os.walk('../kaggle/input'):
    for filename in filenames:
        table_name = filename.split('.')[0]
        table_path = os.path.join(dirname, filename)
        try:
            data[table_name] = pd.read_csv(table_path)
        except UnicodeDecodeError:
            data[table_name] = pd.read_csv(table_path, encoding='cp1252')
        except Exception as e:
            print(f"Error with {filename}: {e}")

# Split dict of dataframes by gender and other (supplemental) data
mens_data = dict()
womens_data = dict()
supplemental_data = dict()

for k, v in data.items():
    if k.startswith("M"):
        mens_data[k] = v
    elif k.startswith("W"):
        womens_data[k] = v
    else:
        supplemental_data[k] = v
        

In [73]:
def get_season_stats(dataset, detailed=False, post_season=False, year=None):
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if detailed:
        if post_season:
            df = dataset[f"{gender}NCAATourneyDetailedResults"]
        else:
            df = dataset[f"{gender}RegularSeasonDetailedResults"]
        
    else:
        if post_season:
            df = dataset[f"{gender}NCAATourneyCompactResults"]
        else:
            df = dataset[f"{gender}RegularSeasonCompactResults"]
    
    if year is not None:
        df = df[df["Season"] == year]
    return df, gender

def compute_margins_of_victory(df):
    df["margin"] = df["WScore"] - df["LScore"]
    
    win_df = df[["WTeamID", "margin"]].rename(columns={"WTeamID": "TeamID"})
    lose_df = df[["LTeamID", "margin"]].rename(columns={"LTeamID": "TeamID"})
    lose_df["margin"] = -lose_df["margin"]

    res = pd.concat([win_df, lose_df], axis=0)
    return res.groupby("TeamID")["margin"].mean()

def join_team_names(df, data, gender="M"):
    """
    df: pd.DataFrame
        dataframe appending teams to
    data: dict[str, pd.DataFrame]
        dictionary of all table names and data
    """
    res = pd.merge(df, data[f"{gender}Teams"][["TeamID", "TeamName"]], on="TeamID")
    return res

def create_srs(df,gender):

    df["margin"] = df["WScore"] - df["LScore"]
    win_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"WTeamID": "team_id", "LTeamID": "opp_id"}
    )
    lose_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"LTeamID": "team_id", "WTeamID": "opp_id"}
    )
    lose_df["margin"] = -lose_df["margin"]

    teams = pd.concat([win_df, lose_df], axis=0)
    spreads = compute_margins_of_victory(df)
    
    terms = []
    solutions = []

    for team_id in spreads.keys():
        row = []
        opps = list(teams[teams["team_id"] == team_id]["opp_id"])

        for opp_id in spreads.keys():
            if opp_id == team_id:
                # coef for the team itself should be 1
                row.append(1)
            elif opp_id in opps:
                # coef for opponents is 1 over num of opps
                row.append(-1.0/len(opps))
            else:
                # teams not faced get a 0 coef
                row.append(0)
        terms.append(row)

        solutions.append(spreads[team_id])

    solutions, _, _, _ = np.linalg.lstsq(np.array(terms), np.array(solutions), rcond=None)
    
    ratings = list(zip( spreads.keys(), solutions ))
    srs = pd.DataFrame(ratings, columns=['team', 'rating'])
    rankings = srs.sort_values('rating', ascending=False).reset_index()[['team', 'rating']]
    rankings = join_team_names(rankings.rename(columns={"team": "TeamID"}), data, gender=gender)
    return rankings

def get_coach_win_perc(
    dataset: dict,
    regular_season: bool,
    year:int = 2024
) -> pd.DataFrame:
    """
    
    parameters
    ----------
    dataset: dict
        dictionary of datasets to use. it will be
        mens_data or womens_data.
        
    year: int
        year to filter data. it will get coaches stats for everything
        up until this year. (model can't have any look ahead bias). for post
        season games, use a year one less than the year of interest.
        
    returns
    -------
    coaches_stats: pd.DataFrame
        dataframe with count of wins, win percentage, and std dev
        of wins.
    """
    
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if regular_season:
        df = dataset[f"{gender}RegularSeasonCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] <= year]
    else:
        df = dataset[f"{gender}NCAATourneyCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] < year]
        
    
    
    winning_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "WTeamID"],
        right_on=["Season", "TeamID"]
    )

    winning_coaches_df = winning_coaches_df[
        (winning_coaches_df['DayNum'] >= winning_coaches_df['FirstDayNum']) 
        & (winning_coaches_df['DayNum'] <= winning_coaches_df['LastDayNum'])
    ]
    winning_coaches_df["win"] = 1

    #Make sure the join dind't create dupes
    assert len(winning_coaches_df) == len(df)

    losing_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "LTeamID"],
        right_on=["Season", "TeamID"]
    )

    losing_coaches_df = losing_coaches_df[
        (losing_coaches_df['DayNum'] >= losing_coaches_df['FirstDayNum']) 
        & (losing_coaches_df['DayNum'] <= losing_coaches_df['LastDayNum'])
    ]
    losing_coaches_df["win"] = 0

    #Make sure the join dind't create dupes
    assert len(losing_coaches_df) == len(df)

    coaches_df = pd.concat(
        [
            losing_coaches_df[["CoachName", "win"]],
            winning_coaches_df[["CoachName", "win"]]
        ],
        axis=0
    )

    coach_stats = (
        coaches_df
        .groupby("CoachName")["win"]
        .describe()
        .sort_values("count", ascending=False)
        [["count", "mean", "std"]]
        .fillna(0)
    )

    return coach_stats
def get_system_ratings(
    mens_dataset, #There are only ratings for men
    systems: List[str],
    year: int=2024,
):
    """
    gets system ratings for each team for specified systems for a specific year.
    
    parameters
    ---------
    mens_dataset: dict
        dictionary of datasets for men
    systems: List[str]
        list of dictionaries we are interested in seeing
    year: int
        year to look for ratings
    moving_average: str
        specifies how to calculate rolling ratings for given systems.
        if None, the system takes the most recent system rating
    
    returns
    -------
    df: pd.DataFrame
        data that reflects ratings for a team
    """
    
    # Filter by season - only take most recent
    df = mens_dataset["MMasseyOrdinals"]
    df = df[df["Season"] == year]
    
    # Filter by system
    df = df[df["SystemName"].isin(systems)]
    
    latest_rank = (
        df
        .sort_values("RankingDayNum")
        .groupby(["TeamID","SystemName"])
        ["OrdinalRank"]
        .last()
        .unstack("SystemName")
        .reset_index().
        rename(columns = {i: i+"_latest" for i in systems})
    )
    
    transformed_df = (
        df
        .sort_values(by="RankingDayNum")
        .groupby(["TeamID", "SystemName"], group_keys=False)
        ["OrdinalRank"]
        .rolling(5) # TODO: Parameterize this (window and moving average method)
        .mean()
        .unstack("SystemName")
        .reset_index()
        .drop("level_1", axis=1)
        .groupby("TeamID")
        [systems]
        .last()
        .reset_index()
        .rename(columns = {i: i+"_rolling" for i in systems})
    )
    
    res = pd.merge(latest_rank, transformed_df, on="TeamID")

    return res

def get_post_season(data, year):
    
    df, gender = get_season_stats(
            data, 
            detailed=False, 
            post_season=True, 
            year=year
    )
    
    # Shuffle teams for positional encoding (model shouldn't have winning teams features as the same)
    df["TeamID"] = np.where(
        np.random.uniform(0,1, size=len(df)) > .5, 
        df["WTeamID"], 
        df["LTeamID"]
    )
    df["team_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["WScore"], 
        df["LScore"]
    )
    df["OppID"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LTeamID"], 
        df["WTeamID"]
    )
    df["opp_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LScore"], 
        df["WScore"]
    )
    df = df.drop(
        ["WTeamID", "LTeamID", "WScore", "LScore", "WLoc", "NumOT"],
        axis=1
    )
    
    return df

def get_features(mens_data, year, systems):
    # Season Stats
    df, gender = get_season_stats(
        mens_data, 
        detailed=False, 
        post_season=False, 
        year=year
    )

    # Rating System
    srs = create_srs(df, gender)

    # System Ratings
    system_ratings = get_system_ratings(
        mens_data, 
        systems=systems
    ) #KenPom, Nolan ELO, EPSN BPI

    # Ratings df
    ratings_df = pd.merge(
                srs,
                system_ratings,
                on="TeamID"
    )

    # Coaches postseason win stats
    coaches_postseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=False, 
        year=year
    ).rename(columns={"count": "count_post", "mean": "mean_post", "std": "std_post"})

    # Coaches regular season win stats
    coaches_regseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=True, 
        year=year
    ).rename(columns={"count": "count_reg", "mean": "mean_reg", "std": "std_reg"})

    coaches_df = pd.merge(
        coaches_regseason_win_df,
        coaches_postseason_win_df,
        on="CoachName",
        how="left"
    ).fillna(0)

    # Get coaches for the year and only grab the most recent coach for a certain team
    curr_coaches = (
        mens_data["MTeamCoaches"][
            mens_data["MTeamCoaches"]["Season"] == year
        ]
        .sort_values("FirstDayNum")
        .groupby("TeamID")["CoachName"]
        .last()
        .reset_index()
    )

    # Get coach stats for current coaches
    coaches_df = pd.merge(
        curr_coaches,
        coaches_df,
        on="CoachName",
        how="left"
    )


    feature_df = (
        pd.merge(
            ratings_df,
            coaches_df
        )
        .drop(["TeamName", "CoachName"], axis=1)
    )

    
    return feature_df


def merge_features_to_games(feature_df, post_season_df, year, training=True):
    
    post_season_merged = pd.merge(
        pd.merge(
            feature_df,
            post_season_df,
            on="TeamID",
        ),
        feature_df,
        left_on="OppID",
        right_on="TeamID",
        suffixes=("_team", "_opp")
    )
    if training:
        post_season_merged["win"] = post_season_merged["team_score"] > post_season_merged["opp_score"]
        post_season_merged = (
            post_season_merged
            .drop(
                ["team_score", "OppID", "opp_score", "DayNum"], 
                axis=1
            )
            .rename(columns = {"TeamID_team": "TeamID", "TeamID_opp": "OppID"})
        )

    for col in post_season_merged.columns:
        if col.replace("_team", "_opp") in post_season_merged.columns and "_team" in col:
            post_season_merged[col.replace("_team", "_diff")] = post_season_merged[col] - post_season_merged[col.replace("_team", "_opp")]
            post_season_merged = post_season_merged.drop([col, col.replace("_team", "_opp")], axis=1)

    post_season_merged = post_season_merged.drop(["Season_x", "Season_y"], axis=1)
    return post_season_merged

In [74]:
#TODO: Add features from box scores

In [75]:
def get_team_stats(df, year=None):

    if year is not None:
        df[df["Season"] == year]
    
    df["margin"] = df["WScore"] - df["LScore"]
    win_df = df.rename(
        columns={"WTeamID": "team_id", "LTeamID": "opp_id", "WLoc": "Loc"}
    )
    win_df = win_df.rename(columns={col: col[1:] + "_opp" for col in win_df.columns if col.startswith("L") and col != "Loc"})
    win_df = win_df.rename(columns={col: col[1:] for col in win_df.columns if col.startswith("W") and not col.endswith("_opp")})
    
    lose_df = df.rename(
        columns={"LTeamID": "team_id", "WTeamID": "opp_id", "WLoc": "Loc"}
    )
    lose_df = lose_df.rename(columns={col: col[1:] for col in lose_df.columns if col.startswith("L") and col != "Loc"})
    lose_df = lose_df.rename(columns={col: col[1:] + "_opp" for col in lose_df.columns if col.startswith("W")})
    lose_df["Loc"] = lose_df["Loc"].apply(lambda x: "H" if x == "A" else "A" if x == "H" else "N")
    lose_df["margin"] = -lose_df["margin"]

    teams = pd.concat([win_df, lose_df], axis=0)

    df = teams.groupby(["Season", "team_id"])[
        ['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast',
        'TO', 'Stl', 'Blk', 'PF', 'FGM_opp', 'FGA_opp', 'FGM3_opp', 'FGA3_opp',
        'FTM_opp', 'FTA_opp', 'OR_opp', 'DR_opp', 'Ast_opp', 'TO_opp',
        'Stl_opp', 'Blk_opp', 'PF_opp', 'margin'
        ]
    ].agg([
            ("mean", "mean"), 
            ("quant25" , lambda x: x.quantile(.25)), 
            ("quant75", lambda x: x.quantile(.75))
        ]
    ).reset_index()
    df.columns = [(col + "_" + agg_func).strip("_") for col, agg_func in zip(df.columns.get_level_values(0), df.columns.get_level_values(1))]

    for col in df.columns:
        if (
            "_opp" in col
            and col.replace("_opp", "") in df.columns 
            and col not in ["Season", "team_id"]
        ):
            new_col = col.replace("_opp", "") + "_diff"
            df[new_col] = df[col.replace("_opp", "")] - df[col]
            df = df.drop([col.replace("_opp", ""), col], axis=1)
    return df

In [76]:
def get_advanced_features(mens_data, year=None, systems=["POM", "NOL", "EBP"]):
    df, _ = get_season_stats(
        mens_data,
        detailed=True,
        post_season=False,
        year=year
    )
    adv_features = get_team_stats(df)
    basic_features = get_features(mens_data=mens_data, year=year, systems=systems)

    res = pd.merge(
        basic_features,
        adv_features.rename(columns={"team_id": "TeamID"}),
        on="TeamID"
    )

    return res

In [77]:
def save_data(train_dict, _dir="baseline"):
    if not os.path.isdir(_dir):
        os.mkdir(_dir)
    for year in tqdm(train_dict):
        train_dict[year].to_csv(f"{_dir}/{year}.csv", index=False)
        
def load_data(_dir="baseline"):
    if not os.path.isdir(_dir):
        raise NotADirectoryError(f"{_dir} is not a directory")
    else:
        train_data = dict()
        for dirname, _, filenames in tqdm(os.walk(_dir)):
            for filename in filenames:
                table_name = filename.split('.')[0]
                table_path = os.path.join(dirname, filename)
                try:
                    train_data[table_name] = pd.read_csv(table_path, index_col=False)
                except UnicodeDecodeError:
                    train_data[table_name] = pd.read_csv(table_path, encoding='cp1252')
                except Exception as e:
                    print(f"Error with {filename}: {e}")
    return train_data

# Model V2
Improvements:
- Build in detailed stats from box scores
- Build in conference stats
- Tune hyperparams
- Add in tempo
- Add in experience (maybe)
- Maybe add round number as a feature

# Baseline Model

Use a simple rating system (SRS) combined with KenPom to fit a model predicting the probability of a given matchup in the tournament. Then randomly sample from the distribution outputed from the model to create multiple submissions.

### Features:
- SRS system from the regular season
- KenPom ranking system
- Coach historical success in post season and regular season

### Model: XGBoost


In [78]:
# This function will change a lot based on what we are trying to predict
# Simplest training method is to grab team ids from previous years and pull in reg season stats to make a prediction
# what we should try to get to is running simulations and making predictions based on matchups then have some sort of loss metric for how good or bad a bracket is.
# Also adding stats like if they're on a run or not would be cool (tough to do at inference time)
def create_mens_training_data():
    
    training_data = dict()
    
    for year in tqdm(range(2003, 2025)):
        
        feature_df = get_advanced_features(mens_data, year=year, systems=["POM", "NOL", "EBP"])
        post_season_df = get_post_season(mens_data, year)
        post_season_merged = merge_features_to_games(feature_df, post_season_df, year)
        
        training_data[year] = post_season_merged
    
    return training_data

In [79]:
# Need to join this on who is in the tournament every year and create predictions based on matchups 2003-2023
train_dict = create_mens_training_data()

100%|██████████| 22/22 [01:46<00:00,  4.86s/it]


In [80]:
save_data(train_dict, "../data/baseline")

100%|██████████| 22/22 [00:00<00:00, 221.13it/s]


In [81]:
train_data=load_data("../data/baseline")

1it [00:00, 22.96it/s]


In [82]:
temp_df = pd.concat(train_dict.values(), ignore_index=True)
# for col in temp_df.columns:
#     if col.replace("_team", "_opp") in temp_df.columns and "_team" in col:
#         temp_df[col.replace("_team", "_diff")] = temp_df[col] - temp_df[col.replace("_team", "_opp")]
#         temp_df = temp_df.drop([col, col.replace("_team", "_opp")], axis=1)

In [90]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(
        pd.merge(
            pd.merge(
                temp_df[temp_df["Season"] == 2023], 
                mens_data["MTeams"], 
                on="TeamID"
            ),
            mens_data["MTeams"], 
            left_on="OppID",
            right_on="TeamID",
            suffixes=("_TEAM", "_OPP")
        )
    )

,TeamID_TEAM,OppID,Season,win,rating_diff,EBP_latest_diff,NOL_latest_diff,POM_latest_diff,POM_rolling_diff,NOL_rolling_diff,EBP_rolling_diff,count_reg_diff,mean_reg_diff,std_reg_diff,count_post_diff,mean_post_diff,std_post_diff,margin_mean_diff,margin_quant25_diff,margin_quant75_diff,FGM_mean_diff_diff,FGM_quant25_diff_diff,FGM_quant75_diff_diff,FGA_mean_diff_diff,FGA_quant25_diff_diff,FGA_quant75_diff_diff,FGM3_mean_diff_diff,FGM3_quant25_diff_diff,FGM3_quant75_diff_diff,FGA3_mean_diff_diff,FGA3_quant25_diff_diff,FGA3_quant75_diff_diff,FTM_mean_diff_diff,FTM_quant25_diff_diff,FTM_quant75_diff_diff,FTA_mean_diff_diff,FTA_quant25_diff_diff,FTA_quant75_diff_diff,OR_mean_diff_diff,OR_quant25_diff_diff,OR_quant75_diff_diff,DR_mean_diff_diff,DR_quant25_diff_diff,DR_quant75_diff_diff,Ast_mean_diff_diff,Ast_quant25_diff_diff,Ast_quant75_diff_diff,TO_mean_diff_diff,TO_quant25_diff_diff,TO_quant75_diff_diff,Stl_mean_diff_diff,Stl_quant25_diff_diff,Stl_quant75_diff_diff,Blk_mean_diff_diff,Blk_quant25_diff_diff,Blk_quant75_diff_diff,PF_mean_diff_diff,PF_quant25_diff_diff,PF_quant75_diff_diff,TeamName_TEAM,FirstD1Season_TEAM,LastD1Season_TEAM,TeamID_OPP,TeamName_OPP,FirstD1Season_OPP,LastD1Season_OPP
0,1104,1394,2023,True,15.328418,-156,-157,-167,-188.2,-169.4,-168.6,197.0,0.047338,-0.020087,8.0,0.444444,0.527046,8.331643,7.00,4.75,2.133874,3.00,2.00,-6.866126,-7.25,-8.75,4.356998,2.75,4.75,11.817444,10.00,15.00,-0.293103,1.00,-1.75,2.424949,0.25,5.00,0.419878,-0.25,-0.25,5.871197,6.25,5.75,4.274848,3.75,4.25,4.555781,2.00,6.00,-3.235294,-3.75,-3.00,3.215010,5.00,2.00,-2.263692,0.00,-5.00,Alabama,1985,2024,1394,TAM C. Christi,2003,2024
1,1369,1394,2023,False,-4.829131,190,183,180,158.6,168.0,176.0,31.0,-0.199623,0.016783,-1.0,0.000000,0.000000,-5.829676,-3.00,-10.00,-1.841170,-4.00,0.00,-2.386625,-3.00,-4.00,0.554859,0.00,0.00,5.213166,5.00,7.00,-2.702194,1.00,-5.00,-1.552769,-1.00,-1.00,-2.344828,-3.00,-3.00,-2.336468,-2.00,-3.00,-0.900731,0.00,-3.00,1.258098,-1.00,1.00,-2.454545,-2.00,-3.00,2.225705,3.00,3.00,0.540230,2.00,-1.00,SE Missouri St,1992,2024,1394,TAM C. Christi,2003,2024
2,1104,1268,2023,True,8.564821,-39,-67,-37,-44.8,-57.0,-51.0,-249.0,0.119129,-0.032421,3.0,0.277778,0.118798,6.464349,7.00,3.75,2.278075,5.00,1.00,-3.024955,-3.25,-4.75,3.226381,2.75,3.75,6.604278,3.00,8.00,-1.318182,-2.00,-2.75,-1.840463,0.25,-4.00,0.673797,-2.25,2.75,5.086453,5.25,5.75,4.902852,3.75,5.25,3.267380,2.00,5.00,-1.992870,-1.75,-2.00,1.110517,3.00,0.00,0.862745,1.00,1.00,Alabama,1985,2024,1268,Maryland,1985,2024
3,1104,1361,2023,False,7.925153,-24,-8,-9,-12.2,-9.2,-30.0,72.0,-0.074898,0.038482,6.0,0.444444,0.527046,6.145221,1.00,6.50,1.511029,3.75,0.75,-4.270221,-3.25,-6.75,4.123162,3.75,5.75,12.215074,9.25,14.25,-1.000000,-3.75,-3.00,-1.670956,-3.75,-2.00,-0.141544,-1.00,-0.25,3.522059,3.00,4.75,2.924632,1.75,4.00,3.426471,1.75,5.25,-2.047794,-1.50,-2.00,0.571691,3.00,-1.00,0.248162,1.00,-1.75,Alabama,1985,2024,1361,San Diego St,1985,2024
4,1158,1361,2023,False,0.801575,74,36,96,105.0,51.0,80.2,138.0,-0.111025,0.051681,-1.0,0.000000,0.000000,5.438447,3.00,6.75,-0.554924,-2.25,0.75,1.239583,4.00,1.00,3.775568,4.00,4.00,12.974432,10.25,13.25,2.772727,1.25,1.75,2.442235,3.00,4.00,2.214962,3.25,2.00,0.284091,-0.25,2.00,-0.311553,-1.00,-0.25,-0.477273,-2.25,0.25,0.793561,1.25,1.00,-1.205492,0.00,-1.00,0.294508,0.00,0.25,Col Charleston,1992,2024,1361,San Diego St,1985,2024
5,1166,1361,2023,False,0.108135,-20,-4,-4,-4.0,-1.6,-25.2,503.0,-0.153495,0.063127,13.0,0.375000,0.500000,0.620265,-5.00,1.75,-1.312500,-3.25,-1.25,-6.063447,-4.00,-6.00,2.108902,2.00,3.00,8.186553,6.25,9.25,1.136364,0.25,-0.25,0.714962,1.00,1.00,-2.027462,-1.75,-3.00,1.375000,2.75,1.00,1.142992,1.00,2.75,2.704545,1.75,3.25,-1.751894,0.25,-2.00,0.552083,2.00,0.00,-0.978220,-2.00,-1.75,Creighton,1985,2024,1361,San Diego St,1985,2024
6,1166,1343,2023,True,5.203994,-41,-19,-52,-50.6,-29.8,-42.2,387.0,-0.017191,0.003852,15.0,0.375000,0.500000,2.077441,-1.50,6.00,-0.444444,-3.50,0.00,-3

In [180]:
all_data = pd.concat(train_data.values(), ignore_index=True)

In [181]:
all_data

,TeamID,OppID,Season,win,rating_diff,EBP_latest_diff,NOL_latest_diff,POM_latest_diff,POM_rolling_diff,NOL_rolling_diff,...,TO_quant75_diff_diff,Stl_mean_diff_diff,Stl_quant25_diff_diff,Stl_quant75_diff_diff,Blk_mean_diff_diff,Blk_quant25_diff_diff,Blk_quant75_diff_diff,PF_mean_diff_diff,PF_quant25_diff_diff,PF_quant75_diff_diff
0,1242,1181,2003,True,6.179431,6,-5,6,2.6,-3.0,...,5.50,1.233333,0.00,2.50,-0.533333,0.75,-0.25,-1.366667,-1.00,-1.50
1,1141,1181,2003,False,-7.139134,271,157,256,249.6,161.2,...,8.75,-3.411494,-3.00,-6.00,0.483908,0.75,2.75,1.209195,-2.00,3.75
2,1242,1112,2003,True,1.310308,11,-2,11,10.8,-3.0,...,-1.25,0.800000,0.00,3.25,-1.321429,-1.00,-1.00,1.021429,0.75,-0.25
3,1323,1112,2003,False,-4.624479,147,166,132,150.0,191.4,...,-1.00,-1.596774,-2.00,-1.25,-0.692396,-2.00,0.00,1.095622,-0.75,0.00
4,1211,1112,2003,False,-7.990716,15,6,17,16.4,13.8,...,0.00,-2.596774,-2.00,-2.25,-0.402074,-1.00,1.00,2.192396,1.75,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,1338,1235,2023,True,-0.767814,28,31,43,50.4,44.4,...,4.75,-1.909091,-2.00,-1.50,0.787879,1.00,0.75,-3.671402,-3.75,-3.00
1310,1277,1425,2023,True,-0.777913,-78,-95,-81,-81.6,-119.2,...,1.50,-1.379032,0.00,-1.75,-2.929435,-2.00,-4.00,1.016129,0.25,2.75
1311,1277,1266,2023,True,-3.375097,4,32,7,5.0,19.0,...,5.50,-4.893738,-5.00,-5.00,-0.506641,-1.00,0.00,-0.542694,-0.25,1.00
1312,1436,1266,2023,False,-4.461264,69,56,86,95.2,59.0,...,2.25,-3.264706,-3.00,-5.00,0.485294,0.00,1.00,-1.652574,-2.25,-0.75


In [231]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

def feature_selection(_df, num_features_out=20):

    # Assuming you have a DataFrame `df` with features and `target` as your target variable
    df = _df.copy()
    target = df['win']
    df = df.drop(["win", "TeamID", "OppID"], axis=1)

    # 1. Variance Threshold
    selector = VarianceThreshold(threshold=0.01)  # Adjust the threshold value as needed
    df_reduced = selector.fit_transform(df)
    selected_columns = df.columns[selector.get_support()]
    print(f"Shape after Variance Threshold: {df_reduced.shape}")

    # 2. Univariate Selection
    k = 50  # Select the top 100 features based on univariate tests
    univariate_selector = SelectKBest(f_classif, k=k)
    df_reduced = univariate_selector.fit_transform(df_reduced, target)
    unscaled_selected_columns = selected_columns[univariate_selector.get_support()]
    print(f"Shape after Univariate Selection: {df_reduced.shape}")

    # 3. Model-based Feature Importance
    # Scale the features
    # scaler = StandardScaler()
    # df_scaled = scaler.fit_transform(df_reduced)

    # Fit a Random Forest model to get feature importance
    rf = RandomForestClassifier()
    rf.fit(df_reduced, target.astype(int))

    # Get feature importances and select the top features
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1][:num_features_out]  # Select top 30 features
    df_final = df_reduced[:, indices]
    selected_columns = unscaled_selected_columns[indices]
    print(f"Shape after Model-based Feature Selection: {df_final.shape}")

    # # 4. Recursive Feature Elimination (Optional, if more reduction is needed)
    # # Note: RFE can be computationally expensive; consider using RFECV for cross-validated selection
    # rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=num_features_out)
    # df_reduced = rfe.fit_transform(df_scaled, target.astype(int))
    # selected_columns = selected_columns[rfe.get_support()]
    # print(f"Shape after RFE: {df_reduced.shape}")

    return df_final, target, selected_columns

In [227]:
all_data.isna().sum().sum()

0

In [228]:
# all_data = all_data[
#     [
#         # "margin_mean_diff", 
#         "Season", 
#         "win",
#         "rating_diff",
#         # 'Blk_quant75_diff_diff', 
#         # 'DR_quant25_diff_diff', 
#         # 'PF_mean_diff_diff', 
#         'FTA_mean_diff_diff', 
#         'FGA_quant25_diff_diff', 
#         'count_post_diff', 
#         'TO_mean_diff_diff',
#         "POM_latest_diff",
#         "POM_rolling_diff"
#     ]
# ]



In [298]:
first_testing_year = 2021

train = all_data[all_data["Season"] < first_testing_year]
test = all_data[all_data["Season"] >= first_testing_year]


# X_train = train.drop(["win","Season"], axis=1)
# y_train = train["win"]
# X_test = test.drop(["win","Season"], axis=1)
# y_test = test["win"]


# training_cols = X_train.columns
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

X_train, y_train, feature_cols = feature_selection(train)
X_test = test.drop(["win","Season", "TeamID", "OppID"], axis=1)
X_test = X_test[feature_cols].values
y_test = test["win"]
training_cols = feature_cols



Shape after Variance Threshold: (1115, 55)
Shape after Univariate Selection: (1115, 50)
Shape after Model-based Feature Selection: (1115, 20)


### XGBoost

In [299]:
# # # model = xgboost.XGBClassifier(n_estimators=500, subsample=.9, early_stopping=True, gamma=1)
# # model = xgboost.XGBClassifier(n_estimators=100, early_stopping=True)
# # # model.fit(X_train, y_train)


# # for year in range(2003, first_testing_year):

# #     train = all_data[all_data["Season"] == year]
# #     if train.empty:
# #         continue

# #     X_train = train.drop("win", axis=1)
# #     y_train = train["win"]

# #     model.fit(X_train, y_train)

# # Define parameters for the XGBoost model
# params = {
#     'objective': 'binary:logistic',
#     'eval_metric': 'logloss',
#     'tree_method': 'gpu_hist',
#     'max_depth': 4,  # Reduced max depth
#     'subsample': 0.8,  # Subsample percentage of the training data
#     'lambda': 2,  # Increased L2 regularization
#     'alpha': 0.5,  # Increased L1 regularization
#     'eta': 0.01,  # Lower learning rate
# }
# validation_years = range(first_testing_year - 2, first_testing_year)

# # Prepare the validation set
# validation_data = all_data[all_data["Season"].isin(validation_years)]
# X_val = validation_data.drop("win", axis=1)
# y_val = validation_data["win"]
# dval = xgboost.DMatrix(X_val, label=y_val)

# model = None

# for year in range(2003, first_testing_year - 2):
#     train = all_data[all_data["Season"] == year]
#     if train.empty:
#         continue

#     X_train = train.drop("win", axis=1)
#     y_train = train["win"]

#     dtrain = xgboost.DMatrix(X_train, label=y_train)

#     # Early stopping requires at least one set to be passed in evals
#     if model is None:
#         model = xgboost.train(params, dtrain, num_boost_round=1000, 
#                           evals=[(dval, 'validation')], early_stopping_rounds=50)
#     else:
#         model = xgboost.train(params, dtrain, num_boost_round=1000, xgb_model=model,
#                           evals=[(dval, 'validation')], early_stopping_rounds=50)

# # After the loop, 'model' will be your trained model


In [300]:
# import xgboost as xgb
# from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
# from sklearn.metrics import roc_auc_score

# # Prepare the validation set outside the objective function
# validation_data = all_data[all_data["Season"].isin(range(first_testing_year - 2, first_testing_year))]
# X_val = validation_data.drop("win", axis=1)
# y_val = validation_data["win"]
# dval = xgb.DMatrix(X_val, label=y_val)

# # Define the parameter space
# space = {
#     'max_depth': hp.choice('max_depth', range(3, 10)),
#     'subsample': hp.uniform('subsample', 0.7, 1),
#     'eta': hp.uniform('eta', 0.01, 0.3),
#     'lambda': hp.uniform('lambda', 0, 2),
#     'alpha': hp.uniform('alpha', 0, 1)
# }

# # Define the objective function to minimize
# def objective(params):
#     global model

#     # Update our parameters with the current set of hyperparameters
#     params['objective'] = 'binary:logistic'
#     params['eval_metric'] = 'logloss'
#     params['tree_method'] = 'gpu_hist'

#     model = None
#     for year in range(2003, first_testing_year - 2):
#         train = all_data[all_data["Season"] == year]
#         if train.empty:
#             continue

#         X_train = train.drop("win", axis=1)
#         y_train = train["win"]

#         dtrain = xgb.DMatrix(X_train, label=y_train)

#         if model is None:
#             model = xgb.train(params, dtrain, num_boost_round=1000, 
#                               evals=[(dval, 'validation')], early_stopping_rounds=50)
#         else:
#             model = xgb.train(params, dtrain, num_boost_round=1000, xgb_model=model,
#                               evals=[(dval, 'validation')], early_stopping_rounds=50)

#     # Evaluate the model on the validation set
#     y_pred = model.predict(dval)

#     # Calculate the loss
#     loss = 1 - roc_auc_score(y_val, y_pred)
#     return {'loss': loss, 'status': STATUS_OK}

# # Run the hyperparameter search using the Tree of Parzen Estimators (TPE) algorithm
# trials = Trials()
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=30, trials=trials)

# print(best)

In [365]:
# BEST PARAMS: {'alpha': 0.12695801173189394, 'eta': 0.010970710588702692, 'lambda': 1.8788973229927652, 'max_depth': 2, 'subsample': 0.8952525703549468}

validation_years = range(first_testing_year - 2, first_testing_year)


iterative_training = False
if iterative_training:
    # Prepare the validation set
    validation_data = all_data[all_data["Season"].isin(validation_years)]
    X_val = validation_data.drop("win", axis=1)
    y_val = validation_data["win"]
    dval = xgboost.DMatrix(X_val, label=y_val)

    model = None

    for year in range(2003, first_testing_year - 2):
        train = all_data[all_data["Season"] == year]
        if train.empty:
            continue

        X_train = train.drop("win", axis=1)
        y_train = train["win"]

        dtrain = xgboost.DMatrix(X_train, label=y_train)

        # Early stopping requires at least one set to be passed in evals
        if model is None:
            model = xgboost.train(params, dtrain, num_boost_round=1000, 
                            evals=[(dval, 'validation')], early_stopping_rounds=50)
        else:
            model = xgboost.train(params, dtrain, num_boost_round=1000, xgb_model=model,
                            evals=[(dval, 'validation')], early_stopping_rounds=50)

else:
    model = xgboost.XGBClassifier(eta=.5, min_child_weight=25, gradient_method="gradient_based")
    # model = xgboost.XGBClassifier(n_estimators=1000, min_child_weight=25)
    model.fit(X_train, y_train)


In [366]:
print(sorted({k: v for k, v in zip(training_cols, model.feature_importances_)}.items(), key=lambda x: -x[1])
)

# # Get feature importance dictionary
# feature_importances = model.get_score(importance_type='weight')

# # Convert to a more interpretable structure (e.g., a sorted list of tuples)
# sorted_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)

# # If you want to print the feature importances
# for feature, importance in sorted_importances:
#     print(f"Feature: {feature}, Importance: {importance}")

 #XGB

# dtest = xgboost.DMatrix(X_test, label=y_test)

train_predicted_output = pd.DataFrame(
    {
        # "Predicted": model.predict_proba(X_test)[:, 1], 
        "Predicted": np.round(model.predict(X_train), 0).astype(int),
        "Actual": y_train.astype(int)
    }
)

predicted_output = pd.DataFrame(
    {
        # "Predicted": model.predict_proba(X_test)[:, 1], 
        "Predicted": np.round(model.predict(X_test), 0).astype(int),
        "Actual": y_test.astype(int)
    }
)

print(f"Train Accuracy: {np.mean(np.where(train_predicted_output['Predicted'] == train_predicted_output['Actual'], 1, 0))}")
print(f"Test Accuracy: {np.mean(np.where(predicted_output['Predicted'] == predicted_output['Actual'], 1, 0))}")

[('rating_diff', 0.28721532), ('NOL_rolling_diff', 0.053740717), ('mean_reg_diff', 0.045097325), ('mean_post_diff', 0.045011792), ('margin_quant25_diff', 0.040672302), ('Stl_mean_diff_diff', 0.039848123), ('POM_latest_diff', 0.038899563), ('count_post_diff', 0.038671777), ('std_post_diff', 0.038563114), ('FGM_mean_diff_diff', 0.037410863), ('Ast_mean_diff_diff', 0.037218627), ('margin_quant75_diff', 0.03702194), ('POM_rolling_diff', 0.036708366), ('margin_mean_diff', 0.036468282), ('EBP_rolling_diff', 0.035822242), ('EBP_latest_diff', 0.03341546), ('NOL_latest_diff', 0.031492412), ('FGM_quant75_diff_diff', 0.029908795), ('count_reg_diff', 0.029207813), ('Stl_quant75_diff_diff', 0.027605228)]
Train Accuracy: 0.9363228699551569
Test Accuracy: 0.6683417085427136


In [367]:
train_predicted_output["Actual"]

0       1
1       0
2       1
3       0
4       0
       ..
1110    1
1111    0
1112    1
1113    1
1114    0
Name: Actual, Length: 1115, dtype: int32

In [368]:
def create_testing_data(df, feature_df, seed_lookup, scaler, year):
    """
    matchup_df: pd.DataFrame
    scaler: StandardScaler
    year: int
    """
    
    matchup_df = df.copy()
    
    matchup_df["TeamID"] = matchup_df["StrongSeed"].apply(
        lambda x: seed_lookup[x]
    )
    matchup_df["OppID"] = matchup_df["WeakSeed"].apply(
        lambda x: seed_lookup[x]
    )
    
    post_season_merged = merge_features_to_games(feature_df, matchup_df, year, training=False)

    post_season_merged = post_season_merged[scaler.feature_names_in_]

    X_test = scaler.transform(post_season_merged)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    return X_test_tensor

def probabilistic_choice(row):
    return np.random.choice([row['StrongSeed'], row['WeakSeed']], p=[row['prob'], 1-row['prob']])

def get_model_predictions(model, inputs, matchup_df, model_type):
    """
    matchup_df: pd.DataFrame
    scaler: StandardScaler
    year: int
    """
    if model_type == "torch":
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Inference mode, gradients not needed
            outputs = model(inputs)
    elif model_type == "xgb":
        outputs = model.predict_proba(inputs)[:, 1]
    
    pred_df = matchup_df.copy()
    pred_df["prob"] = outputs
    pred_df["winner"] = pred_df.apply(probabilistic_choice, axis=1)

    return pred_df


def create_feature_df(year, systems=["POM", "NOL", "EBP"]):
    
    seeding_df = mens_data["MNCAATourneySeeds"][
        mens_data["MNCAATourneySeeds"]["Season"] == year
    ]

    feature_df = get_features(mens_data, year=year, systems=systems)
    
    res = pd.merge(seeding_df, feature_df, on="TeamID")

    return res

# Run Simulation Based on Model
Recursively loop through the seeding df to pick teams at each round.

TODO: Swap out function to use baseline model predictions

In [369]:
def create_submission(model, gender, bracket, year, systems=["POM", "NOL", "EBP"]):

    df = mens_data["MNCAATourneySlots"][
        (mens_data["MNCAATourneySlots"]["Season"] == year)
    ]
    seeding_df = mens_data["MNCAATourneySeeds"][
        mens_data["MNCAATourneySeeds"]["Season"] == year
    ]

    seed_lookup = {
        k: v for k, v in zip(
            seeding_df["Seed"], 
            seeding_df["TeamID"]
        )
    }

    # feature_df = get_features(mens_data, year=year, systems=systems)
    feature_df = get_advanced_features(mens_data=mens_data, year=year)

    def simulate(_df, round_num=0, results=None):
        
        df = _df.copy()
        
        if results is None:
            results = {}
            
        if round_num > 6:
            return results
        
        if round_num == 0: # Play-IN games
            temp_df = df[~df["Slot"].str.startswith("R")]
        
        else:
            temp_df = df[df["Slot"].str.startswith(f"R{round_num}")]
            temp_df["StrongSeed"] = temp_df["StrongSeed"].apply(
                lambda x: results[x] if x in results else x
            )
            temp_df["WeakSeed"] = temp_df["WeakSeed"].apply(
                lambda x: results[x] if x in results else x
            )
            

        inputs = create_testing_data(temp_df, feature_df, seed_lookup, scaler, year)
        temp_df = get_model_predictions(model, inputs, temp_df, model_type="xgb")

        for k, v in zip(temp_df["Slot"], temp_df["winner"]):
            results[k] = v
        
        results = simulate(df, round_num + 1, results)
        return results
    
    round_winner = simulate(df)
    
    df["Team"] = df["Slot"].apply(lambda x: round_winner[x])
    df["Tournament"] = gender
    df["Bracket"] = bracket
    
    return df[["Tournament", "Bracket", "Slot", "Team"]]

In [370]:
create_submission(model=model, gender="M", bracket=1, year=2023)

AttributeError: 'StandardScaler' object has no attribute 'feature_names_in_'

In [72]:
# Add multiprocessing to this function or offload to GPU
brackets = {}
for i in tqdm(range(200)):
    brackets[i] = create_submission(model=model, gender="M", bracket=i, year=2023)

100%|██████████| 200/200 [18:40<00:00,  5.60s/it]


In [19]:
# def worker_function(i):
#     return i, create_submission(model=model, gender="M", bracket=i, year=2023)


# from concurrent.futures import ProcessPoolExecutor, as_completed
# from tqdm.notebook import tqdm as tq

# # Assuming your create_submission function is defined elsewhere and accessible

# def worker_function(i):
#     return create_submission(model="YourModel", gender="M", bracket=i, year=2023)


# def run_parallel():
#     brackets = {}
#     pool_size = multiprocessing.cpu_count()  # Number of processes to create

#     with multiprocessing.Pool(pool_size) as pool:
#         # Map the range of inputs to the worker function
#         # The tqdm call is moved here to track progress of the parallel tasks
#         results = list(tqdm(pool.imap(worker_function, range(100)), total=100))

#     # Populate the brackets dictionary with the results
#     for i, result in results:
#         brackets[i] = result

#     return brackets

# if __name__ == "__main__":
#     # Using a smaller number of processes in Jupyter might be more stable
#     pool_size = min(multiprocessing.cpu_count(), 4)  
#     pool = multiprocessing.Pool(pool_size)
    
#     # Use a list to collect the results
#     results = []
#     for _ in tqdm(pool.imap_unordered(worker_function, range(100)), total=100):
#         results.append(_)
    
#     pool.close()
#     pool.join()

#     # Now process the results
#     brackets = {result['bracket']: result for result in results}

In [73]:
df = pd.concat(brackets.values(), ignore_index=False)

In [74]:
# df.to_csv("../data/submissions/model_v2_5000_runs.csv")

In [75]:
df

,Tournament,Bracket,Slot,Team
2385,M,0,R1W1,W01
2386,M,0,R1W2,W02
2387,M,0,R1W3,W03
2388,M,0,R1W4,W04
2389,M,0,R1W5,W05
...,...,...,...,...
2447,M,199,R6CH,W01
2448,M,199,W16,W16a
2449,M,199,X16,X16b
2450,M,199,Y11,Y11a


In [82]:
def get_successors(seed):
    net = networkx.DiGraph()

    slot_df = mens_data["MNCAATourneySlots"][
        (mens_data["MNCAATourneySlots"]["Season"] == 2023)
    ]

    net.add_edges_from([i for i in zip(slot_df["WeakSeed"].values, slot_df["Slot"].values)])
    net.add_edges_from([i for i in zip(slot_df["StrongSeed"].values, slot_df["Slot"].values)])

    successors = [i[1] for i in dfs_edges(net, seed)]

    return successors

In [84]:
def check_bracket_distribution(df, seed, verbose=False):
    """
    df: pd.DataFrame
        This is the submission dataframe for all brackets
    team: str
        a seed representing the team
    """
    successors = get_successors(seed)
    path_df = df[df["Slot"].isin(successors)]
    path_df = path_df.groupby("Slot")["Team"].apply(lambda x: np.mean(x == seed))

    if verbose:
        seeding_teams = pd.merge(mens_data["MNCAATourneySeeds"][
            mens_data["MNCAATourneySeeds"]["Season"] == 2023
        ], mens_data["MTeams"], on="TeamID")

        try:
            team_name = seeding_teams[seeding_teams["Seed"] == seed]["TeamName"].values[0]
        except:
            team_name=""

        print("*"*50)
        print(f"{team_name} CHANCE TO WIN IT ALL: {path_df[-1]}")
        print("*"*50)

    return path_df

In [87]:
tournament_2023 = pd.merge(
    mens_data["MNCAATourneySeeds"][
        mens_data["MNCAATourneySeeds"]["Season"] == 2023
    ], 
    mens_data["MTeams"], 
    on="TeamID"
)

In [88]:
tournament_2023["round_win_prob"]= tournament_2023["Seed"].apply(lambda x: check_bracket_distribution(df, x)[:6].values)

tournament_2023[
    [
        "round_of_32",
        "sweet16",
        "elite8",
        "final4",
        "final",
        "champ",
    ]
] = list(tournament_2023["round_win_prob"])

In [89]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also

    display(tournament_2023.sort_values(by=["champ", "final", "final4", "elite8", "sweet16", "round_of_32"], ascending=False))

,Season,Seed,TeamID,TeamName,FirstD1Season,LastD1Season,round_win_prob,round_of_32,sweet16,elite8,final4,final,champ
0,2022,W01,1124,Baylor,1985,2024,"[0.965, 0.88, 0.74, 0.64, 0.535, 0.355]",0.965,0.880,0.740,0.640,0.535,0.355
34,2022,Y01,1242,Kansas,1985,2024,"[0.99, 0.57, 0.315, 0.235, 0.215, 0.195]",0.990,0.570,0.315,0.235,0.215,0.195
3,2022,W04,1417,UCLA,1985,2024,"[1.0, 0.93, 0.165, 0.16, 0.14, 0.11]",1.000,0.930,0.165,0.160,0.140,0.110
17,2022,X01,1211,Gonzaga,1985,2024,"[0.985, 0.565, 0.275, 0.245, 0.08, 0.075]",0.985,0.565,0.275,0.245,0.080,0.075
53,2022,Z03,1397,Tennessee,1985,2024,"[0.565, 0.225, 0.095, 0.055, 0.045, 0.04]",0.565,0.225,0.095,0.055,0.045,0.040
54,2022,Z04,1228,Illinois,1985,2024,"[0.9, 0.375, 0.14, 0.095, 0.06, 0.03]",0.900,0.375,0.140,0.095,0.060,0.030
51,2022,Z01,1112,Arizona,1985,2024,"[0.93, 0.45, 0.265, 0.155, 0.07, 0.025]",0.930,0.450,0.265,0.155,0.070,0.025
52,2022,Z02,1437,Villanova,1985,2024,"[0.87, 0.275, 0.165, 0.085, 0.06, 0.02]",0.870,0.275,0.165,0.085,0.060,0.020
21,2022,X05,1163,Connecticut,1985,2024,"[0.96, 0.365, 0.17, 0.13, 0.025, 0.015]",0.960,0.365,0.170,0.130,0.025,0.015
45,2022,Y12,1350,Richmond,1985,2024,"[0.76, 0.265, 0.11, 0.08, 0.02, 0.015]",0.760,0.265,0.110,0.080,0.020,0.015


In [37]:
test

,rating_team,EBP_latest_team,NOL_latest_team,POM_latest_team,POM_rolling_team,NOL_rolling_team,EBP_rolling_team,count_reg_team,mean_reg_team,std_reg_team,...,POM_rolling_opp,NOL_rolling_opp,EBP_rolling_opp,count_reg_opp,mean_reg_opp,std_reg_opp,count_post_opp,mean_post_opp,std_post_opp,win
1247,20.255540,1,2,1,1.0,3.0,1.0,893.0,0.687570,0.463744,...,5.8,15.8,6.8,587.0,0.664395,0.472604,27.0,0.592593,0.500712,True
1248,18.432270,6,12,7,6.8,10.8,4.8,256.0,0.691406,0.462818,...,51.6,67.8,55.8,505.0,0.572277,0.495239,6.0,0.166667,0.408248,True
1249,17.149414,5,4,6,5.6,6.4,6.0,1111.0,0.665167,0.472145,...,29.6,18.0,24.6,139.0,0.589928,0.493625,0.0,0.000000,0.000000,False
1250,15.476536,106,81,95,94.4,77.2,109.2,632.0,0.697785,0.459582,...,20.6,22.6,19.8,750.0,0.850667,0.356655,60.0,0.633333,0.485961,False
1251,15.390988,19,14,21,20.6,22.6,19.8,750.0,0.850667,0.356655,...,60.0,31.0,61.8,332.0,0.617470,0.486739,4.0,0.000000,0.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,2.134960,218,254,260,250.6,234.2,213.2,108.0,0.324074,0.470210,...,43.4,63.4,43.2,581.0,0.736661,0.440824,30.0,0.633333,0.490133,False
1310,-0.380099,264,262,281,275.8,268.6,259.2,92.0,0.391304,0.490716,...,15.0,5.8,15.8,934.0,0.770878,0.420493,76.0,0.723684,0.450146,False
1311,-1.725279,352,352,354,353.6,348.2,349.4,90.0,0.444444,0.499688,...,195.0,180.2,173.4,59.0,0.644068,0.482905,1.0,0.000000,0.000000,False
1312,-3.448262,300,287,324,327.2,283.2,305.6,32.0,0.531250,0.507007,...,277.2,263.2,275.0,644.0,0.537267,0.498997,7.0,0.285714,0.487950,True
